# Paolina functions
For development and testing of IC-based Paolina

In [1]:
%matplotlib inline

from __future__ import print_function
import abc
import pylab
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.patches import Ellipse

from   collections import namedtuple
from   invisible_cities.database import load_db
from   invisible_cities.core.system_of_units_c import units

import random
import tables as tb
import numpy as np
import tensorflow as tf
import networkx as nx

NSIPM = 1792

------
## Test events
Read in using old PMap format, converted to a format to be passed to reconstruction method.

In [ ]:
nfile = "/Users/jrenner/IFIC/jerenner/ICARO/invisible_cities/utils/hdf5_NEXT_NEW_se_1M_v0_08_07_0.h5"
erange_low = 0
erange_high = 10

In [ ]:
# plot a 48x48 SiPM map
# -- carried over from NEW_kr_diff_mc_train.ipynb
def plot_test_event(l_X,l_Y,l_Q):
    """
    Plots a SiPM map in the NEW Geometry
    """

    # set up the figure
    fig = plt.figure();
    ax1 = fig.add_subplot(111);
    fig.set_figheight(10.0)
    fig.set_figwidth(10.0)
    ax1.axis([-250, 250, -250, 250]);

    # plot the SiPM pattern
    for xx,yy,qq in zip(l_X,l_Y,l_Q):
        r = Ellipse(xy=(xx, yy), width=2., height=2.);
        r.set_facecolor('0');
        r.set_alpha(qq);
        ax1.add_artist(r);

    # place a large blue circle for actual EL points
    #if(hasattr(l_X0, "__len__")):
    #    for xx,yy in zip(l_X0,l_Y0):
    #        mrk = Ellipse(xy=(xx,yy), width=4., height=4.)
    #        mrk.set_facecolor('b')
    #        ax1.add_artist(mrk)
    #else:
    #    mrk = Ellipse(xy=(l_X0,l_Y0), width=4., height=4.)
    #    mrk.set_facecolor('b')
    #    ax1.add_artist(mrk)
        
    # place a large red circle for reconstructed points
    #if(ept is not None):
    #    xpt = ept[0]*fscale - fshift*fscale
    #    ypt = ept[1]*fscale - fshift*fscale
    #    mrk = Ellipse(xy=(xpt,ypt), width=2., height=2.);
    #    mrk.set_facecolor('r');
    #    ax1.add_artist(mrk);
        
    plt.xlabel("x (mm)");
    plt.ylabel("y (mm)");

In [ ]:
# load the SiPM (x,y) values
DataSensor = load_db.DataSiPM()
xs = DataSensor.X.values
ys = DataSensor.Y.values

# create (x,y) lists
_sipm_x = np.zeros(NSIPM)
_sipm_y = np.zeros(NSIPM)
for ID, x, y in zip(range(NSIPM), xs, ys):
    _sipm_x[ID] = x
    _sipm_y[ID] = y

In [ ]:
nevts = erange_high - erange_low

# open the hdf5 file containing the PMaps.
fname = "{0}".format(nfile)
print("Opening file: {0}".format(fname))
fpmap = tb.open_file(fname,'r')
pmaps = fpmap.root.PMaps.PMaps
print(pmaps)

# set up the lists
tbl_T = []; tbl_E = [] 
tbl_X = []; tbl_Y = []; tbl_Q = []

# loop over all events.
rnum = 0                    # row number in table iteration
ev = 0                      # processed event number (starts at 0 and runs to nevts-1)
evtnum = pmaps[0]['event']  # event number from file
while(rnum < pmaps.nrows and ev < nevts):

    # Skip rows corresponding to event numbers outside of given range.
    if((evtnum < erange_low or evtnum > erange_high) and not (erange_low < 0 or erange_high < 0)):
        rnum += 1
        if(rnum < pmaps.nrows):
            evtnum = pmaps[rnum]['event']
        continue

    # Attempt to get all times, cathode energies, and anode values
    #  for one event.
    times = []
    cathodes = []
    anodes = []
    while(rnum < pmaps.nrows and pmaps[rnum]['event'] == evtnum):
        if(pmaps[rnum]['signal'] == b'S2'):
            times.append(pmaps[rnum]['time'])
            cathodes.append(pmaps[rnum]['cathode'])
            anodes.append(pmaps[rnum]['anode'])
        rnum += 1
        
    # convert to numpy arrays
    anodes = np.array(anodes)
    cathodes = np.array(cathodes)
    times = np.array(times)

    # if we had an S2 for this event, get the relevant information and add the event
    if(len(times) > 0):
        print("Had S2")
        
        # store the energies and times of each projection
        tbl_E.append(cathodes)
        tbl_T.append(times)
        
        # create lists of (x,y,q) for each projection and store
        evt_X = []; evt_Y = []; evt_Q = []
        for aa in anodes:
            ids = np.nonzero(aa)
            evt_X.append(_sipm_x[ids])
            evt_Y.append(_sipm_y[ids])
            evt_Q.append(aa[ids])
        tbl_X.append(evt_X)
        tbl_Y.append(evt_Y)
        tbl_Q.append(evt_Q)

    # set to the next event
    if(rnum < pmaps.nrows):
        print("Processed event {0} with rnum {1} of {2} rows.".format(evtnum, rnum, pmaps.nrows))
        evtnum = pmaps[rnum]['event']
        
# convert to arrays
tbl_X = np.array(tbl_X)
tbl_Y = np.array(tbl_Y)
tbl_Q = np.array(tbl_Q)

In [ ]:
# plot a test projection
t_evt = 0; t_proj = 14
t_X = tbl_X[t_evt][t_proj]; t_Y = tbl_Y[t_evt][t_proj]
t_Q = tbl_Q[t_evt][t_proj]
t_Q /= np.max(t_Q)

plot_test_event(t_X,t_Y,t_Q)

-------
# Reconstruction
Reconstruct 1 event (produce hit collection).  Later Paolina can be run on the hit collection.

In [ ]:
# event to be reconstructed
r_evt = 4

In [ ]:
# reconstruction objects (already defined)
class Event:
    def __init__(self):
        self.evt  = -1
        self.time = -1

    def __str__(self):
        s = "{0}Event\n{0}".format("#"*20 + "\n")
        for attr in self.__dict__:
            s += "{}: {}\n".format(attr, getattr(self, attr))
        return s

    def copy(self, other):
        assert isinstance(other, Event)
        for attr in other.__dict__:
            setattr(self, attr, copy.deepcopy(getattr(other, attr)))

    @abc.abstractmethod
    def store(self, *args, **kwargs):
        pass

class Hit:
    def __init__(self):
        self.Npeak = -1
        self.X     = -1e12
        self.Y     = -1e12
        self.R     = -1e12
        self.Phi   = -1e12
        self.Z     = -1
        self.E     = -1
        self.Q     = -1
        self.Nsipm = -1

class HitCollection(list, Event):
    def __init__(self):
        list .__init__(self)
        Event.__init__(self)

    def store(self, row):
        for hit in self:
            row["event"] = self.evt
            row["time" ] = self.time

            row["npeak"] = hit.Npeak
            row["X"    ] = hit.X
            row["Y"    ] = hit.Y
            row["Z"    ] = hit.Z
            row["R"    ] = hit.R
            row["Phi"  ] = hit.Phi
            row["Nsipm"] = hit.Nsipm
            row["Q"    ] = hit.Q
            row["E"    ] = hit.E
            row["Ecorr"] = hit.Ecorr

            row.append()

# cluster named tuple (make the class using the namedtuple function)
Cluster = namedtuple('Cluster', 'Q X Y Xrms Yrms Nsipm')

In [ ]:
# from xy_algorithms.py (Gonzalo)
def barycenter(xs, ys, qs, default=np.nan):
    q    = np.sum(qs)
    n    = len(qs)
    x    = np.average(xs, weights=qs)         if n and q>0 else default
    y    = np.average(ys, weights=qs)         if n and q>0 else default
    xvar = np.sum(qs * (xs - x)**2) / (q - 1) if n and q>0 else default
    yvar = np.sum(qs * (ys - y)**2) / (q - 1) if n and q>0 else default

    c    = Cluster(q, x, y, xvar**0.5, yvar**0.5, n)
    return c

In [ ]:
def reco_algorithm(xs, ys, Qs, rmax=30*units.mm, T=3.5*units.pes):
    """
    rmax is the maximum radius of a cluster
    T is the threshold for local maxima (this kwarg may be unnecessary)
    returns a list of Clusters
    """
    c = []
    xs = np.copy(xs)
    ys = np.copy(ys)
    qs = np.copy(Qs)

    # While there are more local maxima
    while len(qs) > 0:
        i_max = np.argmax(qs)    # SiPM with largest Q
        if qs[i_max] < T: break  # largest Q remaining is negligible

        # get SiPMs within rmax of SiPM with largest Q
        dists = np.sqrt((xs - xs[i_max]) ** 2 + (ys - ys[i_max]) ** 2)
        cluster = np.where(dists < rmax)[0]

        # get barycenter of this cluster
        c.append(barycenter(xs[cluster], ys[cluster], qs[cluster]))

        xs = np.delete(xs, cluster) # delete the SiPMs
        ys = np.delete(ys, cluster) # contributing to
        qs = np.delete(qs, cluster) # this cluster

    return c

In [ ]:
# different than Gonzalo's because we're using our test setup
def compute_xy_position(xs,ys,Qs):
    return reco_algorithm(xs, ys, Qs)

In [ ]:
# split the energy across the clusters
def split_energy(Es, clusters):
    qtot = np.sum([cl.Q for cl in clusters])
    return [(Es*cl.Q/qtot) for cl in clusters]

In [ ]:
# reconstruct the entire event
e_X = tbl_X[r_evt]
e_Y = tbl_Y[r_evt]
e_T = tbl_T[r_evt]
e_E = tbl_E[r_evt]
e_Q = tbl_Q[r_evt]
print("Total energy is {0}".format(np.sum(e_E)))

# set up the HitCollection
hitc = HitCollection()
hitc.evt   = r_evt
hitc.time  = 0

# iterate through all slices
for Xs,Ys,Ts,Es,Qs in zip(e_X,e_Y,e_T,e_E,e_Q):
    clusters = compute_xy_position(Xs,Ys,Qs)
    es       = split_energy(Es, clusters)
    z        = (Ts - 0.0) * 1000.0 / 1000.0  #(t_slice - S1t) * units.ns * self.drift_v
    for c, e in zip(clusters, es):
        hit       = Hit()
        hit.Npeak = 0 #npeak
        hit.X     = c.X
        hit.Y     = c.Y
        hit.R     = (c.X**2 + c.Y**2)**0.5
        hit.Phi   = np.arctan2(c.Y, c.X)
        hit.Z     = z
        hit.Q     = c.Q
        hit.E     = e
        hit.Ecorr = e #correct_energy(e, c.X, c.Y, z)
        hit.Nsipm = c.Nsipm
        hitc.append(hit)

---------
# Paolina algorithm
At this point the hit collection for an event should be in the variable `hitc`.

In [ ]:
# variables we need to set initially
vol_min = np.array([-250, -250, 0])  # volume minimum (x,y,z)
vol_max = np.array([250, 250, 800])  # volume maximum (x,y,z)
vox_size = np.array([10, 10, 10])    # voxel size
blob_radius = 15.                    # blob radius in mm

In [ ]:
# define voxel object
class Voxel:
    def __init__(self,ID,X,Y,Z,E,ix,iy,iz,size_x,size_y,size_z,tID):
        self.ID     = ID
        self.X      = X
        self.Y      = Y
        self.Z      = Z
        self.E      = E
        self.ix     = ix
        self.iy     = iy
        self.iz     = iz
        self.size_x = size_x
        self.size_y = size_y
        self.size_z = size_z
        self.iID    = tID
#Voxel = namedtuple('Voxel', 'ID X Y Z E ix iy iz size_x size_y size_z tID')

In [ ]:
def build_voxels(hitc):
    """Builds a list of voxels from the specified hit collection.
    """
    
    # calculate the size of the volume dimensions
    vdim = vol_max - vol_min
    
    # create the voxel array
    varr = np.zeros([vdim[0],vdim[1],vdim[2]])
    
    # add the energy of all hits to the voxels
    for hh in hitc:
        ivox = int((hh.X - vol_min[0]) / vox_size[0])
        jvox = int((hh.Y - vol_min[1]) / vox_size[1])
        kvox = int((hh.Z - vol_min[2]) / vox_size[2])
        varr[ivox][jvox][kvox] += hh.E

    # get lists of the nonzero x,y,z indices and E values
    nzx,nzy,nzz = np.nonzero(varr)
    nze = varr[np.nonzero(varr)]
    
    # create voxel objects
    voxelc = []; vid = 0
    for ix,iy,iz,ee in zip(nzx,nzy,nzz,nze):
        voxelc.append(Voxel(vid,
                            vol_min[0] + ix*vox_size[0],
                            vol_min[1] + iy*vox_size[1],
                            vol_min[2] + iz*vox_size[2],
                            ee, ix, iy, iz,
                            vox_size[0],vox_size[1],vox_size[2],
                            -1))
        #print("Added voxel {0},{1},{2} with {3}".format(ix,iy,iz,ee))
        vid += 1
        
    return voxelc

In [ ]:
def calc_adj_matrix(voxelc):
    """Creates the adjacency matrix.
        -1         --> self
        0          --> not a neighbor
        (distance) --> voxels are neighbors    
    """
    # use the voxels: determine neighboring voxels by face, edge, or corner connections
    adj_mat = np.zeros([len(voxelc),len(voxelc)])

    # iterate through all voxels, and for each one find the neighboring voxels
    for vv1 in voxelc:
        for vv2 in voxelc:
            if(vv1.ix == vv2.ix and vv1.iy == vv2.iy and vv1.iz == vv2.iz):
                adj_mat[vv1.ID][vv2.ID] = -1.
            elif ((vv1.ix == vv2.ix+1 or vv1.ix == vv2.ix-1 or vv1.ix == vv2.ix) and 
                  (vv1.iy == vv2.iy+1 or vv1.iy == vv2.iy-1 or vv1.iy == vv2.iy) and 
                  (vv1.iz == vv2.iz+1 or vv1.iz == vv2.iz-1 or vv1.iz == vv2.iz)):
                adj_mat[vv1.ID][vv2.ID] = np.sqrt((vv2.X-vv1.X)**2 + (vv2.Y-vv1.Y)**2 + (vv2.Z-vv1.Z)**2)
        
    return adj_mat

In [ ]:
def construct_tracks(voxelc,adj_mat):
    """Constructs all independent tracks given the list of voxels and adjacency matrix.
        Note: assumes the rows and columns of the adjacency matrix correspond
            to the voxels in the order that they are placed in voxelc
    """
    # add all voxels as nodes to a Graph
    pgraph = nx.Graph()
    pgraph.add_nodes_from(voxelc)
        
    # add edges connecting each node to its neighbor nodes based on the values in the adjacency matrix
    for nA in voxelc:
        for nB in voxelc:
            ndist = adj_mat[nA.ID][nB.ID]
            if(ndist > 0):
                #print("-- Adding edge from {0} to {1} with weighting of {2}".format(nA.ID,nB.ID,ndist))
                pgraph.add_edge(nA,nB,weight=ndist)
    
    # find all independent tracks
    trks = []
    while(pgraph.number_of_nodes() > 0):
        
        # add all nodes with a path from node 0 to a single track
        tnodes = []; tid = 0; gnodes = pgraph.nodes()
        for nn in gnodes:
            if(nx.has_path(pgraph,gnodes[0],nn)):
                nn.tID = tid
                tnodes.append(nn)
                tid += 1
        tgraph = nx.Graph()
        tgraph.add_nodes_from(tnodes)
        tgraph.add_weighted_edges_from(pgraph.edges(tnodes,data='weight'))
        trks.append(tgraph)
        
        # remove these nodes from the original graph and start again
        pgraph.remove_nodes_from(tnodes)
        
    # find the largest independent track
    itmax = np.argmax([trk.number_of_nodes() for trk in trks])
    #print("Found {0} tracks with max having {1} nodes".format(len(trks),trks[itmax].number_of_nodes()))
    
    return (itmax,trks)

In [ ]:
def calc_dist_mat(tgraph):
    """Calculates the distance matrix and longest shortest path.
    """
    
    # initialize the distance matrix
    tvoxelc = tgraph.nodes()
    dist_mat = np.zeros([len(tvoxelc),len(tvoxelc)])
    dmax = -1.; v1max = None; v2max = None
    
    # compute the matrix, using only nodes in the specified track
    for n1,vv1 in enumerate(tvoxelc):
        for vv2 in tvoxelc[0:n1]:

            # calculate the length of the shortest path between these two voxels
            dist = nx.astar_path_length(tgraph,vv1,vv2)
            #print("--- Adding dist of {0}".format(dist))
            dist_mat[vv1.tID][vv2.tID] = dist_mat[vv2.tID][vv1.tID] = dist
            if(dist > dmax or dmax < 0):
                dmax = dist; v1max = vv1; v2max = vv2
                

    # compute one longest shortest path
    print("Longest shortest path is of length {0}".format(dist_mat[v1max.tID][v2max.tID]))
    spath = nx.astar_path(tgraph,v1max,v2max)
    
    return (dist_mat,spath)

In [ ]:
def construct_blobs(tgraph,dist_mat,spath):
    """Construct the blobs.
    """
    tvoxelc = tgraph.nodes()
    Eblob1 = Eblob2 = 0
    ext1 = spath[0]; ext2 = spath[-1]
    #print("found ext1 {0} and ext2 {1}".format(ext1,ext2))
    
    # add the energies of voxels within 1 blob radius of each extreme
    for vv in tvoxelc:
        dist1 = dist_mat[ext1.tID][vv.tID]
        if(dist1 < blob_radius):
            Eblob1 += vv.E
        dist2 = dist_mat[ext2.tID][vv.tID]
        if(dist2 < blob_radius):
            Eblob2 += vv.E

    return (Eblob1,Eblob2)

In [ ]:
def paolina(hitc):
    """The Paolina reconstruction algorithm.
    
        returns:
        voxelc - a list of voxels in the track
        spath - the longest shortest path (as a sequence of voxels) in the track
        Eblob1 - the energy of the blob constructed about the first voxel in spath
        Eblob2 - the energy of the blob constructed about the last voxel in spath
    """
    voxelc = build_voxels(hitc)                                  # voxelize the event
    adj_mat = calc_adj_matrix(voxelc)                            # calculate the adjacency matrix
    itmax,trks = construct_tracks(voxelc,adj_mat)                # construct the tracks
    dist_mat, spath = calc_dist_mat(trks[itmax])                 # calculate distance matrix and longest shortest path
    Eblob1, Eblob2 = construct_blobs(trks[itmax],dist_mat,spath) # construct the blobs
    
    # return key quantities
    return (voxelc,trks,itmax,spath,Eblob1,Eblob2)

In [ ]:
# perform the Paolina algorithm
voxelc,trks,itmax,spath,Eblob1,Eblob2 = paolina(hitc)
print("Total of {0} tracks; longest contains {1} voxels".format(len(trks),len(spath)))
print("Blobs: {0} and {1}".format(Eblob1,Eblob2))

In [ ]:
# plot the voxelized tracks
# 2D histogram
fig = plt.figure(3);
fig.set_figheight(5.0);
fig.set_figwidth(20.0);

varr_x = []; varr_y = []; varr_z = []; varr_c = []
for vv in voxelc:
    varr_x.append(vv.X)
    varr_y.append(vv.Y)
    varr_z.append(vv.Z)
    varr_c.append(vv.E)

vtrk_max = np.array([np.max(varr_x),np.max(varr_y),np.max(varr_z)])
vtrk_min = np.array([np.min(varr_x),np.min(varr_y),np.min(varr_z)])

# create the x-y projection
ax1 = fig.add_subplot(131);
hxy, xxy, yxy = np.histogram2d(varr_y, varr_x, weights=varr_c, normed=False, bins=(1.0*(vol_max[1]-vol_min[1])/vox_size[1], 1.0*(vol_max[0]-vol_min[0])/vox_size[0]), range=[[vol_min[1],vol_max[1]],[vol_min[0],vol_max[0]]])
extent1 = [yxy[0], yxy[-1], xxy[0], xxy[-1]]
sp1 = ax1.imshow(hxy, extent=extent1, interpolation='none', aspect='auto', origin='lower', cmap='jet')
ax1.set_xlabel("x (mm)")
ax1.set_ylabel("y (mm)")
cbp1 = plt.colorbar(sp1)
cbp1.set_label('Energy (Q)')

# Create the y-z projection.
ax2 = fig.add_subplot(132);
hyz, xyz, yyz = np.histogram2d(varr_z, varr_y, weights=varr_c, normed=False, bins=(1.0*(vol_max[2]-vol_min[2])/vox_size[2], 1.0*(vol_max[1]-vol_min[1])/vox_size[1]), range=[[vol_min[2],vol_max[2]],[vol_min[1],vol_max[1]]])
extent2 = [yyz[0], yyz[-1], xyz[0], xyz[-1]]
sp2 = ax2.imshow(hyz, extent=extent2, interpolation='none', aspect='auto', origin='lower', cmap='jet')
ax2.set_xlabel("y (mm)")
ax2.set_ylabel("z (mm)")
cbp2 = plt.colorbar(sp2);
cbp2.set_label('Energy (Q)');

# Create the x-z projection.
ax3 = fig.add_subplot(133);
hxz, xxz, yxz = np.histogram2d(varr_z, varr_x, weights=varr_c, normed=False, bins=(1.0*(vol_max[2]-vol_min[2])/vox_size[2], 1.0*(vol_max[0]-vol_min[0])/vox_size[0]), range=[[vol_min[2],vol_max[2]],[vol_min[0],vol_max[0]]])
extent3 = [yxz[0], yxz[-1], xxz[0], xxz[-1]]
sp3 = ax3.imshow(hxz, extent=extent3, interpolation='none', aspect='auto', origin='lower', cmap='jet')
ax3.set_xlabel("x (mm)")
ax3.set_ylabel("z (mm)")
cbp3 = plt.colorbar(sp3);
cbp3.set_label('Energy (Q)');